activation checkpointing 

https://pytorch.org/docs/stable/checkpoint.html

In [ ]:
import torch
import sklearn

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits


In [ ]:
x,y=sklearn.datasets.load_digits(return_X_y=True)
x=torch.tensor(x/16).float().cuda() # FP32
y=torch.tensor(y).long().cuda()
print(x.shape,x.dtype)
print(y.shape,y.dtype)

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self,input_size,hidden_sizes,output_size):
        super(MLP,self).__init__()
        self.fc_first=torch.nn.Linear(input_size, hidden_sizes[0])
        fc_middle=[]
        for i in range(1,len(hidden_sizes)-1):
            fc_middle.append(torch.nn.Linear(hidden_sizes[i-1],hidden_sizes[i]))
            fc_middle.append(torch.nn.ReLU())
        self.fc_middle=torch.nn.Sequential(*fc_middle)
        self.fc_final=torch.nn.Linear(hidden_sizes[-1], output_size)
    
    def forward(self,x,checkpoint=False):
        out=self.fc_first(x)
        out=torch.relu(out)
        if checkpoint:
            out=torch.utils.checkpoint.checkpoint(lambda x:self.fc_middle(x),out,use_reentrant=False)
        else:
            out=self.fc_middle(out)
        out=self.fc_final(out)
        return out

checkpoint：消耗18MB

In [ ]:
model=MLP(input_size=64,hidden_sizes=[256,128,128],output_size=10).cuda()
loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

iter=0
while True:
    optimizer.zero_grad()

    out=model(x,checkpoint=True)
    loss=loss_fn(out,y)
    loss.backward()
    optimizer.step()

    iter+=1
    if iter%100000==0:
        print(f'iter={iter} loss={loss.item()} cuda_mem={torch.cuda.memory_allocated()}Bytes')
    if loss.item()<=1e-3:
        break

no checkpoint：消耗20MB

In [ ]:
model=MLP(input_size=64,hidden_sizes=[256,128,128],output_size=10).cuda()
loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

iter=0
while True:
    optimizer.zero_grad()

    out=model(x,checkpoint=False)
    loss=loss_fn(out,y)
    loss.backward()
    optimizer.step()

    iter+=1
    if iter%100000==0:
        print(f'iter={iter} loss={loss.item()} cuda_mem={torch.cuda.memory_allocated()}Bytes')
    if loss.item()<=1e-3:
        break